In [0]:
from keras.models import Sequential
from keras.layers import Dense
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Cargamos los datos de training.

In [0]:
dataset = pd.read_sas('/content/tablatransportestrain.sas7bdat',format='sas7bdat')
dataset

,sid,date,hour,x_origin_0,y_origin_0,termicSensation_origin,x_destination_0,y_destination_0,termicSensation_destination,transport_mode,distance,price,eta,min_distance,min_price,min_eta,binaryTarget
0,b'1000014',2018-11-12,17.0,116.35,39.98,49.65,116.39,39.99,49.65,2.0,5736.0,300.0,1875.0,5736.0,0.0,1454.0,1.0
1,b'1000014',2018-11-12,17.0,116.35,39.98,49.65,116.39,39.99,49.65,6.0,6158.0,0.0,1857.0,5736.0,0.0,1454.0,0.0
2,b'1000063',2018-11-12,15.0,116.49,39.91,58.75,116.54,39.93,58.92,1.0,6644.0,200.0,2988.0,6404.0,0.0,1362.0,1.0
3,b'1000063',2018-11-12,15.0,116.49,39.91,58.75,116.54,39.93,58.92,3.0,6663.0,0.0,1362.0,6404.0,0.0,1362.0,0.0
4,b'1000068',2018-11-12,12.0,116.53,39.77,53.62,116.52,39.81,53.54,1.0,9842.0,200.0,3037.0,6614.0,0.0,877.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
348216,b'999855',2018-11-12,16.0,116.37,39.95,56.86,116.40,39.94,56.95,1.0,3151.0,2000.0,2109.0,2899.0,0.0,875.0,1.0
348217,b'999855',2018-11-12,16.0,116.37,39.95,56.86,116.40,39.94,56.95,1.0,2925.0,200.0,2039.0,2899.0,0.0,875.0,1.0
348218,b'999855',2018-11-12,16.0,116.37,39.95,56.86,116.40,39.94,56.95,3.0,3808.0,0.0,997.0,2899.0,0.0,875.0,0.0
348219,b'999855',2018-11-12,16.0,116.37,39.95,56.86,116.40,39.94,56.95,5.0,2941.0,0.0,2568.0,2899.0,0.0,875.0,0.0


Eliminamos la variable *sid*.

In [0]:
dataset = dataset.drop(columns='sid')
dataset

,date,hour,x_origin_0,y_origin_0,termicSensation_origin,x_destination_0,y_destination_0,termicSensation_destination,transport_mode,distance,price,eta,min_distance,min_price,min_eta,binaryTarget
0,2018-11-12,17.0,116.35,39.98,49.65,116.39,39.99,49.65,2.0,5736.0,300.0,1875.0,5736.0,0.0,1454.0,1.0
1,2018-11-12,17.0,116.35,39.98,49.65,116.39,39.99,49.65,6.0,6158.0,0.0,1857.0,5736.0,0.0,1454.0,0.0
2,2018-11-12,15.0,116.49,39.91,58.75,116.54,39.93,58.92,1.0,6644.0,200.0,2988.0,6404.0,0.0,1362.0,1.0
3,2018-11-12,15.0,116.49,39.91,58.75,116.54,39.93,58.92,3.0,6663.0,0.0,1362.0,6404.0,0.0,1362.0,0.0
4,2018-11-12,12.0,116.53,39.77,53.62,116.52,39.81,53.54,1.0,9842.0,200.0,3037.0,6614.0,0.0,877.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
348216,2018-11-12,16.0,116.37,39.95,56.86,116.40,39.94,56.95,1.0,3151.0,2000.0,2109.0,2899.0,0.0,875.0,1.0
348217,2018-11-12,16.0,116.37,39.95,56.86,116.40,39.94,56.95,1.0,2925.0,200.0,2039.0,2899.0,0.0,875.0,1.0
348218,2018-11-12,16.0,116.37,39.95,56.86,116.40,39.94,56.95,3.0,3808.0,0.0,997.0,2899.0,0.0,875.0,0.0
348219,2018-11-12,16.0,116.37,39.95,56.86,116.40,39.94,56.95,5.0,2941.0,0.0,2568.0,2899.0,0.0,875.0,0.0


In [0]:
dataset.binaryTarget = dataset.binaryTarget.apply(lambda x: int(x)) # Pasar a entero variable target
dataset.transport_mode = dataset.transport_mode.apply(lambda x: int(x)) # Pasar a entero variable modo transporte

dataset.dtypes

date                           datetime64[ns]
hour                                  float64
x_origin_0                            float64
y_origin_0                            float64
termicSensation_origin                float64
x_destination_0                       float64
y_destination_0                       float64
termicSensation_destination           float64
transport_mode                          int64
distance                              float64
price                                 float64
eta                                   float64
min_distance                          float64
min_price                             float64
min_eta                               float64
binaryTarget                            int64
dtype: object

Eliminamos los datos que hemos considerado como missings.

In [0]:
dataset = dataset.drop([50550,50551,50552],axis=0)

In [0]:
dataset['weekday_number'] = dataset['date'].dt.dayofweek
dataset['weekday_number'].describe()
dataset['weekday_number'].unique()
dataset

,date,hour,x_origin_0,y_origin_0,termicSensation_origin,x_destination_0,y_destination_0,termicSensation_destination,transport_mode,distance,price,eta,min_distance,min_price,min_eta,binaryTarget,weekday_number
0,2018-11-12,17.0,116.35,39.98,49.65,116.39,39.99,49.65,2,5736.0,300.0,1875.0,5736.0,0.0,1454.0,1,0
1,2018-11-12,17.0,116.35,39.98,49.65,116.39,39.99,49.65,6,6158.0,0.0,1857.0,5736.0,0.0,1454.0,0,0
2,2018-11-12,15.0,116.49,39.91,58.75,116.54,39.93,58.92,1,6644.0,200.0,2988.0,6404.0,0.0,1362.0,1,0
3,2018-11-12,15.0,116.49,39.91,58.75,116.54,39.93,58.92,3,6663.0,0.0,1362.0,6404.0,0.0,1362.0,0,0
4,2018-11-12,12.0,116.53,39.77,53.62,116.52,39.81,53.54,1,9842.0,200.0,3037.0,6614.0,0.0,877.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
348216,2018-11-12,16.0,116.37,39.95,56.86,116.40,39.94,56.95,1,3151.0,2000.0,2109.0,2899.0,0.0,875.0,1,0
348217,2018-11-12,16.0,116.37,39.95,56.86,116.40,39.94,56.95,1,2925.0,200.0,2039.0,2899.0,0.0,875.0,1,0
348218,2018-11-12,16.0,116.37,39.95,56.86,116.40,39.94,56.95,3,3808.0,0.0,997.0,2899.0,0.0,875.0,0,0
348219,2018-11-12,16.0,116.37,39.95,56.86,116.40,39.94,56.95,5,2941.0,0.0,2568.0,2899.0,0.0,875.0,0,0


In [0]:
dataset['laboral'] = dataset['weekday_number'] 

dataset.loc[dataset['laboral']==0,'laboral'] = 1
dataset.loc[dataset['laboral']==1,'laboral'] = 1
dataset.loc[dataset['laboral']==2,'laboral'] = 1
dataset.loc[dataset['laboral']==3,'laboral'] = 1
dataset.loc[dataset['laboral']==4,'laboral'] = 1
dataset.loc[dataset['laboral']==5,'laboral'] = 0
dataset.loc[dataset['laboral']==6,'laboral'] = 0

dataset


,date,hour,x_origin_0,y_origin_0,termicSensation_origin,x_destination_0,y_destination_0,termicSensation_destination,transport_mode,distance,price,eta,min_distance,min_price,min_eta,binaryTarget,weekday_number,laboral
0,2018-11-12,17.0,116.35,39.98,49.65,116.39,39.99,49.65,2,5736.0,300.0,1875.0,5736.0,0.0,1454.0,1,0,1
1,2018-11-12,17.0,116.35,39.98,49.65,116.39,39.99,49.65,6,6158.0,0.0,1857.0,5736.0,0.0,1454.0,0,0,1
2,2018-11-12,15.0,116.49,39.91,58.75,116.54,39.93,58.92,1,6644.0,200.0,2988.0,6404.0,0.0,1362.0,1,0,1
3,2018-11-12,15.0,116.49,39.91,58.75,116.54,39.93,58.92,3,6663.0,0.0,1362.0,6404.0,0.0,1362.0,0,0,1
4,2018-11-12,12.0,116.53,39.77,53.62,116.52,39.81,53.54,1,9842.0,200.0,3037.0,6614.0,0.0,877.0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
348216,2018-11-12,16.0,116.37,39.95,56.86,116.40,39.94,56.95,1,3151.0,2000.0,2109.0,2899.0,0.0,875.0,1,0,1
348217,2018-11-12,16.0,116.37,39.95,56.86,116.40,39.94,56.95,1,2925.0,200.0,2039.0,2899.0,0.0,875.0,1,0,1
348218,2018-11-12,16.0,116.37,39.95,56.86,116.40,39.94,56.95,3,3808.0,0.0,997.0,2899.0,0.0,875.0,0,0,1
348219,2018-11-12,16.0,116.37,39.95,56.86,116.40,39.94,56.95,5,2941.0,0.0,2568.0,2899.0,0.0,875.0,0,0,1


In [0]:
dataset['hora_punta'] = 0
index = dataset.loc[(dataset['hour']==7.) | (dataset['hour']==8.) | (dataset['hour']==9.) | (dataset['hour']==17.) | (dataset['hour']==18.) | (dataset['hour']==19.)].index
dataset.loc[index,'hora_punta'] = 1
dataset

,date,hour,x_origin_0,y_origin_0,termicSensation_origin,x_destination_0,y_destination_0,termicSensation_destination,transport_mode,distance,price,eta,min_distance,min_price,min_eta,binaryTarget,weekday_number,laboral,hora_punta
0,2018-11-12,17.0,116.35,39.98,49.65,116.39,39.99,49.65,2,5736.0,300.0,1875.0,5736.0,0.0,1454.0,1,0,1,1
1,2018-11-12,17.0,116.35,39.98,49.65,116.39,39.99,49.65,6,6158.0,0.0,1857.0,5736.0,0.0,1454.0,0,0,1,1
2,2018-11-12,15.0,116.49,39.91,58.75,116.54,39.93,58.92,1,6644.0,200.0,2988.0,6404.0,0.0,1362.0,1,0,1,0
3,2018-11-12,15.0,116.49,39.91,58.75,116.54,39.93,58.92,3,6663.0,0.0,1362.0,6404.0,0.0,1362.0,0,0,1,0
4,2018-11-12,12.0,116.53,39.77,53.62,116.52,39.81,53.54,1,9842.0,200.0,3037.0,6614.0,0.0,877.0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
348216,2018-11-12,16.0,116.37,39.95,56.86,116.40,39.94,56.95,1,3151.0,2000.0,2109.0,2899.0,0.0,875.0,1,0,1,0
348217,2018-11-12,16.0,116.37,39.95,56.86,116.40,39.94,56.95,1,2925.0,200.0,2039.0,2899.0,0.0,875.0,1,0,1,0
348218,2018-11-12,16.0,116.37,39.95,56.86,116.40,39.94,56.95,3,3808.0,0.0,997.0,2899.0,0.0,875.0,0,0,1,0
348219,2018-11-12,16.0,116.37,39.95,56.86,116.40,39.94,56.95,5,2941.0,0.0,2568.0,2899.0,0.0,875.0,0,0,1,0


In [0]:
dataset['llovizna'] = 0

index = dataset.loc[(dataset['date']=='2018-10-08') |
         (dataset['date']=='2018-10-15') |
         (dataset['date']=='2018-10-16') |
         (dataset['date']=='2018-10-22') |
         (dataset['date']=='2018-10-25') |
         (dataset['date']=='2018-11-03') |
         (dataset['date']=='2018-11-04')].index

dataset.loc[index,'llovizna'] = 1
target = dataset.binaryTarget
dataset = dataset.drop(columns='binaryTarget')
dataset['binaryTarget'] = target
dataset = dataset.drop(columns='date')
dataset.columns

Index(['hour', 'x_origin_0', 'y_origin_0', 'termicSensation_origin',
       'x_destination_0', 'y_destination_0', 'termicSensation_destination',
       'transport_mode', 'distance', 'price', 'eta', 'min_distance',
       'min_price', 'min_eta', 'weekday_number', 'laboral', 'hora_punta',
       'llovizna', 'binaryTarget'],
      dtype='object')

In [0]:
data_train = pd.read_csv('/content/Datos_Train80.csv',sep=';')
data_val = pd.read_csv('/content/Datos_Val20.csv',sep=';')

In [0]:
data_train = data_train[data_train['termicSensation_origin'].isna()==False]
data_train = data_train.drop(columns='Unnamed: 0')
data_val = data_val[data_val['termicSensation_origin'].isna()==False]
data_val = data_val.drop(columns='Unnamed: 0')

In [0]:
data_train.dtypes
# data_val.dtypes

hour                             int64
termicSensation_origin         float64
termicSensation_destination    float64
transport_mode                   int64
distance                         int64
price                            int64
eta                              int64
min_distance                     int64
min_price                        int64
min_eta                          int64
weekday_number                   int64
hora_punta                       int64
llovizna                         int64
binaryTarget                     int64
dtype: object

In [0]:
data_train
# data_val

,hour,termicSensation_origin,termicSensation_destination,transport_mode,distance,price,eta,min_distance,min_price,min_eta,weekday_number,hora_punta,llovizna,binaryTarget
0,17,49.65,49.65,6,6158,0,1857,5736,0,1454,0,1,0,0
1,15,58.75,58.92,1,6644,200,2988,6404,0,1362,0,0,0,1
2,15,58.75,58.92,3,6663,0,1362,6404,0,1362,0,0,0,0
3,12,53.62,53.54,1,9842,200,3037,6614,0,877,0,0,0,0
4,12,53.62,53.54,3,8496,0,877,6614,0,877,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
278525,7,31.81,31.69,7,11481,600,3214,9876,0,2330,0,1,0,0
278526,7,31.81,31.69,9,11307,400,2330,9876,0,2330,0,1,0,0
278527,16,56.86,56.95,1,3151,2000,2109,2899,0,875,0,0,0,1
278528,16,56.86,56.95,3,3808,0,997,2899,0,875,0,0,0,0


Definimos las variables explicativas y la variable objetivo.

In [0]:
var_exp_train = data_train.iloc[:,0:12]
target_train = data_train.iloc[:,13]

var_exp_val = data_val.iloc[:,0:12]
target_val = data_val.iloc[:,13]


# Modelo 1

Definimos la red neuronal.

In [0]:
model = Sequential()
model.add(Dense(12,input_dim=12, kernel_initializer='uniform',activation='relu'))
model.add(Dense(8,kernel_initializer='uniform',activation='relu'))
model.add(Dense(1,kernel_initializer='uniform',activation='sigmoid'))

Compilamos el modelo.



In [0]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [0]:
model.fit(var_exp_train,target_train,batch_size=10,epochs=7)

Epoch 1/7
278530/278530 [==============================] - 39s 140us/step - loss: 0.4250 - acc: 0.7931
Epoch 2/7
278530/278530 [==============================] - 39s 139us/step - loss: 0.4324 - acc: 0.7921
Epoch 3/7
278530/278530 [==============================] - 38s 135us/step - loss: 0.4338 - acc: 0.7901
Epoch 4/7
278530/278530 [==============================] - 38s 138us/step - loss: 0.4213 - acc: 0.7943
Epoch 5/7
278530/278530 [==============================] - 40s 142us/step - loss: 0.4231 - acc: 0.7942
Epoch 6/7
278530/278530 [==============================] - 39s 142us/step - loss: 0.4265 - acc: 0.7945
Epoch 7/7
278530/278530 [==============================] - 39s 140us/step - loss: 0.4208 - acc: 0.7944


Evaluamos el modelo.

In [0]:
scores = model.evaluate(var_exp_val,target_val)
print('\n%s: %.2f%%'%(model.metrics_names[1],scores[1]*100))

69688/69688 [==============================] - 1s 19us/step

acc: 79.21%


In [0]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 12)                156       
_________________________________________________________________
dense_8 (Dense)              (None, 8)                 104       
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 9         
Total params: 269
Trainable params: 269
Non-trainable params: 0
_________________________________________________________________


Si quisiéramos predecir:

In [0]:
print (model.predict(training_data).round())

Guardamos este modelo.

In [0]:
# serializar el modelo a JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serializar los pesos a HDF5
model.save_weights("model.h5")
print("¡Modelo guardado!")

¡Modelo guardado!


Si quisiéramos usarlo de nuevo:

In [0]:
# Cargar json y crear el modelo
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# Cargar pesos al nuevo modelo
loaded_model.load_weights("model.h5")
print("Cargado modelo desde disco.")
 
# Compilar modelo cargado y listo para usar.
loaded_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

NameError: ignored

# Modelo 2

In [0]:
from keras.layers import Dropout

In [0]:
model2 = Sequential()
model2.add(Dense(12,input_dim=12, kernel_initializer='random_normal',activation='relu'))
model2.add(Dense(12,kernel_initializer='random_normal',activation='relu'))
model2.add(Dense(8,kernel_initializer='random_normal',activation='relu'))
model2.add(Dense(1,kernel_initializer='random_normal',activation='sigmoid'))

In [0]:
model2.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [0]:
model2.fit(var_exp_train,target_train,batch_size=10,epochs=10)

Epoch 1/10
278530/278530 [==============================] - 44s 157us/step - loss: 0.4498 - acc: 0.7903
Epoch 2/10
278530/278530 [==============================] - 44s 157us/step - loss: 0.4209 - acc: 0.7956
Epoch 3/10
278530/278530 [==============================] - 44s 157us/step - loss: 0.4140 - acc: 0.7995
Epoch 4/10
278530/278530 [==============================] - 43s 154us/step - loss: 0.4186 - acc: 0.7979
Epoch 5/10
278530/278530 [==============================] - 44s 158us/step - loss: 0.4208 - acc: 0.7973
Epoch 6/10
278530/278530 [==============================] - 45s 160us/step - loss: 0.4157 - acc: 0.7983
Epoch 7/10
278530/278530 [==============================] - 42s 150us/step - loss: 0.4186 - acc: 0.7968
Epoch 8/10
278530/278530 [==============================] - 42s 150us/step - loss: 0.4248 - acc: 0.7946
Epoch 9/10
278530/278530 [==============================] - 42s 152us/step - loss: 0.4216 - acc: 0.7946
Epoch 10/10
278530/278530 [==============================] - 42s

In [0]:
scores2 = model2.evaluate(var_exp_val,target_val)
print('\n%s: %.2f%%'%(model2.metrics_names[1],scores2[1]*100))

69688/69688 [==============================] - 2s 22us/step

acc: 78.86%


# Modelo 3

In [0]:
model4 = Sequential()
model4.add(Dense(12,input_dim=12, kernel_initializer='glorot_normal',activation='relu'))
model4.add(Dense(6,kernel_initializer='uniform',activation='relu'))
model4.add(Dense(1,kernel_initializer='glorot_normal',activation='sigmoid'))

In [0]:
model4.compile(loss='binary_crossentropy',optimizer='sgd',metrics=['accuracy'])

In [0]:
model4.summary()

Model: "sequential_21"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_85 (Dense)             (None, 12)                156       
_________________________________________________________________
dense_86 (Dense)             (None, 6)                 78        
_________________________________________________________________
dense_87 (Dense)             (None, 1)                 7         
Total params: 241
Trainable params: 241
Non-trainable params: 0
_________________________________________________________________


In [0]:
model4.fit(var_exp_train,target_train,batch_size=10,epochs=10)

Epoch 1/10
278530/278530 [==============================] - 42s 150us/step - loss: 1.3507 - acc: 0.7876
Epoch 2/10
278530/278530 [==============================] - 41s 148us/step - loss: 0.5175 - acc: 0.7877
Epoch 3/10
 82230/278530 [=======>......................] - ETA: 28s - loss: 0.5173 - acc: 0.7874

In [0]:
keras.Initializers.initializers()

NameError: ignored